<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# myGPTlab

This app helps to process texts (longform content) with ChatGPT via API.

## Start:
**Run all cells - that's it.**
- The notebooks checks automatically, if an initial setup (with PIP etc.) is necessary. The setup status is saved in the file 'installation.done'
- You can force the setup by deleting this file or by going to Seciton "Setup and Configuration" and checking "inital_setup_mode". Afterwards uncheck "Setup and Configuration".

## Working with myGPTlab
Lorem ipsum...


# Setup and Configuration

In [ ]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = False # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [ ]:
folders = {
    'audio': 'audio',
    'audio-chunks': 'audio/chunks',
    'transcript':'audio-transcript',
    'transcript-chunks': 'audio-transcript/chunks',
    'text-input': 'text-input',
    'text-input-backup': 'text-input-backup'
}

In [ ]:
from ipywidgets import widgets
from IPython.display import Javascript, display, clear_output
notify_output = widgets.Output()
display(notify_output)
@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))
#popup('Hello World!')

In [ ]:
import os
if initial_setup_mode != True:
  if os.path.exists('installation.done'):
      initial_setup_mode = False
      print('No initial setup - forced by existing file "installation.done"')
  else:
    initial_setup_mode = True
    print('Starting automatic setup - forced by missing file "installation.done".\n\nEnter API Keys as JSON (in next notebook cell).')
    popup('Starting automatic setup. Enter API Keys as JSON (in next notebook cell).')
else:
  print('Starting setup.\n\nEnter API Keys as JSON (in next notebook cell).')
  popup('Starting setup. Enter API Keys as JSON (in next notebook cell).')

In [ ]:
if initial_setup_mode == True:
  #popup('Enter API Keys as JSON:')
  !wget -q bit.ly/aknip-colab-setup
  %run aknip-colab-setup
else:
  print('No initial setup.')

In [ ]:
creds = json.loads(os.getenv('CREDS'))
key = creds['OpenAI']['v2']['credential']
# print(key)

In [ ]:
if ios_mode == False:
  print('Mac')
else:
  print('iOS')

In [ ]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 -q
else:
  print('No initial setup.')


In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  !pip install gradio -q
else:
  print('No initial setup / iOS.')

In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  %load_ext gradio
else:
  print('No initial setup / iOS.')

In [ ]:
if initial_setup_mode == True:
  f= open('installation.done','w+')
  f.close()
  print('Initial setup done. Application starting.')
  popup('Initial setup done. Application starting.')
else:
  print('No initial setup.')

# Helper Functions

- **create_file_directory**: Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.

In [ ]:
# v2 - 06.08.2023
import shutil
import os

def create_file_directory(directory, always_delete=False):
  # Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.
  # Examples:
  # - create_file_directory('texts', False) => creates a new directory only if it not exists yet
  # - create_file_directory('texts', True) => always deletes existing directory and creates a new one
  if os.path.exists(directory):
    if always_delete:
      # delete the diectory recursively
      shutil.rmtree(directory)
  # create directory
  if not os.path.exists(directory):
    os.mkdir(directory)


def find_files(path, extensions=[".txt"], recursive=False):
    # Recursively (optional) find all files with extension in path
    my_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if extensions == []:
                my_files.append(os.path.join(root, f))
            else:
                for ext in extensions:
                    if f.endswith(ext):
                        my_files.append(os.path.join(root, f))
        # no recursion / don't look inside any subdirectory
        if recursive == False:
            break
    return my_files


def merge_textfiles(path, extensions=[".txt"], recursive=False, new_filename='merged.txt'):
    # Recursively (optional) find all files with extension in path
    my_files = find_files(path, extensions, recursive)
    merged_text = ''
    for filename in my_files:
      # print(filename)
      f= open(filename,'r')
      if f.mode == 'r':
            contents =f.read()
      f.close()
      merged_text = merged_text + contents + '\n\n\n'

    f= open(new_filename,'w+')
    f.write(merged_text)
    f.close()

# The App

In [ ]:
if ios_mode == False:
  import gradio as gr

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 700px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      project_name = gr.Textbox(label="Project name")

      #
      # 1. Input Text
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")

        def text_save(text, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          f= open(proj_name + '/' +  folders['text-input'] + '/input_text.txt','w+')
          f.write(text)
          f.close()
          log_text = "Text saved."
          return log_text
        text_button = gr.Button("Save text")
        text_button.click(text_save, [text_input, project_name], text_output)

        gr.Markdown("")
        gr.Markdown("")

        # Input text via upload
        gr.Markdown("### Or upload your text:")
        upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        file_output = gr.Textbox(label="Result")

        def upload_file(my_file, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          # copy to project directory
          full_upload_path = my_file.name
          just_the_filename = os.path.basename(full_upload_path)
          full_text_path = "./" + proj_name + '/' + folders['text-input'] + '/' + just_the_filename
          shutil.copyfile(full_upload_path, full_text_path)
          # check if file is empty
          f= open(full_text_path,'r')
          if f.mode == 'r': contents =f.read()
          f.close()
          log_text = just_the_filename + "\n"
          if len(contents) == 0:
            log_text = log_text + "Error: Upload file lengt 0 bytes"
          else:
            log_text = log_text + "Upload successful"
          return log_text
        upload_button.upload(upload_file, [upload_button, project_name], file_output)

      #
      # 2. Download full project
      #
      with gr.Tab("Download"):
        gr.Markdown("Download full project as ZIP file.")
        download_button = gr.Button("Download project")
        download_output = gr.File()

        def download_do(proj_name):
          full_text_path = "./" + proj_name
          shutil.make_archive('archive', 'zip', full_text_path)
          result = "Downloading " + full_text_path
          return "archive.zip"
        download_button.click(download_do, project_name, download_output)

      #
      # 3. xxx
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        def greet(name):
          result = "HALLO " + name + "!!!"
          return result
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=True, debug=debug_mode)

else:
  print('iOS Mode - Nothing to do.')